In [ ]:
import json
import pandas as pd
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def parse_data(file):
    for l in open(file,'r'):
        yield json.loads(l)

data = list(parse_data('/content/Sarcasm_Headlines_Dataset_v2.json'))
df = pd.DataFrame(data)
df


In [ ]:
headlines = df["headline"]
df['model_truth_values'] = df['is_sarcastic'].apply(lambda input: [0,1] if input == 1 else [1,0] )
# df['is_sarcastic']
df

,is_sarcastic,headline,article_link,model_truth_values
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...,"[0, 1]"
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...,"[1, 0]"
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...,"[1, 0]"
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...,"[0, 1]"
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...,"[0, 1]"
...,...,...,...,...
28614,1,jews to celebrate rosh hashasha or something,https://www.theonion.com/jews-to-celebrate-ros...,"[0, 1]"
28615,1,internal affairs investigator disappointed con...,https://local.theonion.com/internal-affairs-in...,"[0, 1]"
28616,0,the most beautiful acceptance speech this week...,https://www.huffingtonpost.com/entry/andrew-ah...,"[1, 0]"
28617,1,mars probe destroyed by orbiting spielberg-gat...,https://www.theonion.com/mars-probe-destroyed-...,"[0, 1]"


In [ ]:
sarcastic_headlines = [i['headline']  for i in data if i['is_sarcastic'] == 1 ]
not_sarcastic_headlines = [i['headline']  for i in data if i['is_sarcastic'] == 0 ]

In [ ]:
pd.DataFrame(not_sarcastic_headlines)

,0
0,dem rep. totally nails why congress is falling...
1,eat your veggies: 9 deliciously different recipes
2,my white inheritance
3,5 ways to file your taxes with less stress
4,lots of parents know this scenario
...,...
14980,what our grieving family needs from loved ones...
14981,stephen colbert attempts to list everything tr...
14982,bakery owner vows to stop making wedding cakes...
14983,how san antonio's dominant defense is fueling ...


In [ ]:
all_headlines = df.headline.values
all_headlines
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.3MB 9.3MB/s 
     |████████████████████████████████| 901kB 41.8MB/s 
     |████████████████████████████████| 3.3MB 52.0MB/s 
     |████████████████████████████████| 1.2MB 6.6MB/s 


In [ ]:
# import torch
# xlmr = torch.hub.load('pytorch/fairseq', 'xlmr.large')
# xlmr.eval()

In [ ]:

from transformers import XLMRobertaTokenizer, XLMRobertaModel
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
print(tokenizer)
model = XLMRobertaModel.from_pretrained("xlm-roberta-base")

inputs = tokenizer("Hello world!", return_tensors="pt")
outputs = model(**inputs)


PreTrainedTokenizer(name_or_path='xlm-roberta-base', vocab_size=250002, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
inputs

{'input_ids': tensor([[    0, 35378,  8999,    38,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [ ]:
outputs['last_hidden_state'].shape

torch.Size([1, 5, 768])

In [ ]:
inputs['input_ids'].shape

torch.Size([1, 5])

In [ ]:
outputs['last_hidden_state'][0,1,:].shape

torch.Size([768])

In [ ]:
def get_BERT_input(headlines, tokenizer):
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []
  encoded_dict = tokenizer.batch_encode_plus(
                      headlines,                      # Sentence to encode.
                      add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                      max_length = 65,           # Pad & truncate all sentences.
                      pad_to_max_length = True,
                      return_attention_mask = True,   # Construct attn. masks.
                      return_tensors = 'pt',     # Return pytorch tensors.
                )
  input_ids, attention_mask = encoded_dict['input_ids'], encoded_dict['attention_mask']
  # Add the encoded sentence to the list.    
  input_ids = encoded_dict['input_ids']
  
  # And its attention mask (simply differentiates padding from non-padding).
  attention_mask = encoded_dict['attention_mask']
  return torch.tensor(input_ids), torch.tensor(attention_mask)

In [ ]:
get_BERT_input(["this is a plain text"], tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from

(tensor([[     0,    903,     83,     10, 111719,   7986,      2,      1,      1,
               1,      1,      1,      1,      1,      1,      1,      1,      1,
               1,      1,      1,      1,      1,      1,      1,      1,      1,
               1,      1,      1,      1,      1,      1,      1,      1,      1,
               1,      1,      1,      1,      1,      1,      1,      1,      1,
               1,      1,      1,      1,      1,      1,      1,      1,      1,
               1,      1,      1,      1,      1,      1,      1,      1,      1,
               1,      1]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))

#Dataset

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

class SarcasmDataset(Dataset):
    def __init__(self, df):
        self.headlines = df["headline"]
        self.model_truth_values = df["model_truth_values"]

    def __len__(self):
        return len(self.headlines)

    def __getitem__(self, idx):
        return self.headlines.iloc[idx], torch.tensor(self.model_truth_values.iloc[idx])

In [ ]:
dataset = SarcasmDataset(df)

In [ ]:
dataset[3]

('inclement weather prevents liar from getting to work', tensor([0, 1]))

In [ ]:
train_dataloader = DataLoader(dataset, batch_size=32, shuffle=True) 

In [ ]:
train_dataloader

#Model


In [ ]:
from torch import nn
from torch.optim import AdamW
class SarcasmModel(nn.Module):
    def __init__(self):
      super(SarcasmModel, self).__init__()
      self.XLM = XLMRobertaModel.from_pretrained('xlm-roberta-base')
      self.hidden_1 = nn.Linear(768, 128)
      self.hidden_2 = nn.Linear(128, 2)
      #self.hidden_3 = nn.Linear(16,2)
      self.softMax = nn.Softmax(dim=1)
      self.to_delete = 2

    def forward(self, batch_input_ids, batch_attention_mask):
      bert_output = self.XLM(batch_input_ids, batch_attention_mask)
      # print(batch_input_ids.shape)
      # print(batch_attention_mask.shape)
      hidden_state = bert_output['last_hidden_state']
      sentence_vector = torch.mean(hidden_state, dim = 1)
      #print(sentence_vector.shape)
      x = self.hidden_1(sentence_vector)
      x = self.hidden_2(x)
     # x = self.hidden_3(x)
      probabilities = self.softMax(x)
      #print(probabilities)
      return probabilities





model = SarcasmModel()
model = model.to(device)



Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#Training loop

In [ ]:
loss_function = nn.BCELoss().to(device)
optimizer = AdamW(model.parameters(),
                 lr=2e-5,
                 eps=1e-8)
batch_size = 32
for step, (batch_headlines, batch_model_truth_values) in enumerate(train_dataloader):  #(inputs, outputs)
  batch_model_truth_values = batch_model_truth_values.to(device)
  optimizer.zero_grad()
  batch_input_ids, batch_attention_mask = get_BERT_input(batch_headlines, tokenizer)
  batch_prediction = model(batch_input_ids.to(device), batch_attention_mask.to(device))
  loss = loss_function(batch_prediction, batch_model_truth_values.float())
  model.zero_grad()
  loss.backward()
  optimizer.step()


  if step % 10 == 0:
      iteration = step*batch_size
      print("Iteration:", step*batch_size, "Loss:", loss.data)
      batch_accuracy = torch.mean(torch.sum(batch_prediction * batch_model_truth_values.to(device), dim=1))
      print("Batch Accuracy:", batch_accuracy.data*100)
      if iteration % 5120 == 0:
        # torch.save(model.state_dict(), expt_folder + "SarcasmModel.pt")
        print("Saved Model")
  #print(loss)
  # print(batch_input_ids.shape)
  # print(batch_attention_mask.shape)
  # print(step)
  # print(batch_headlines[0])
  # print(batch_model_truth_values[0])
  if step % 10 == 0:
    print(loss)



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Iteration: 0 Loss: tensor(0.0983, device='cuda:0')
Batch Accuracy: tensor(91.7406, device='cuda:0')
Saved Model
tensor(0.0983, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Iteration: 320 Loss: tensor(0.1245, device='cuda:0')
Batch Accuracy: tensor(92.2663, device='cuda:0')
tensor(0.1245, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Iteration: 640 Loss: tensor(0.0436, device='cuda:0')
Batch Accuracy: tensor(96.1400, device='cuda:0')
tensor(0.0436, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Iteration: 960 Loss: tensor(0.2543, device='cuda:0')
Batch Accuracy: tensor(90.6785, device='cuda:0')
tensor(0.2543, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Iteration: 1280 Loss: tensor(0.1923, device='cuda:0')
Batch Accuracy: tensor(88.3871, device='cuda:0')
tensor(0.1923, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Iteration: 1600 Loss: tensor(0.1078, device='cuda:0')
Batch Accuracy: tensor(96.6537, device='cuda:0')
tensor(0.1078, device='cud

In [ ]:
def predict(b_headline, tokenizer, model):
  batch_input_ids, batch_attention_mask = get_BERT_input(batch_headlines, tokenizer)
  batch_predictions = model(batch_input_ids.to(device), batch_attention_mask.to(device))
  sarcastic_probability = batch_predictions.data[0][1].item() * 100
  not_sarcastic_probability = batch_predictions.data[0][0].item() * 100
  print_string = "Sarcastic:", f'{sarcastic_probability:.2f}', "Not Sarcastic:", f'{not_sarcastic_probability:.2f}'
  # return b_predictions
  return print_string 

predict(["հայաստանում լիքը փող կա"], tokenizer, model)



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


('Sarcastic:', '96.46', 'Not Sarcastic:', '3.54')